# 04-8: Translation from Spanish to German

## Translation from Spanish to German using `transformers.pipeline` (simple way)

In [ ]:
# TODO: Implement translation from Spanish to German using `pipeline`

## Translation from Spanish to German using `AutoTokenizer`, `AutoModelForSeq2SeqLM`

In [1]:
# TODO: Implement translation from Spanish to German using `AutoTokenizer`, `AutoModelForSeq2SeqLM`


## Finetuning OPUS-MT for translation with wmt-16 dataset

In [ ]:
# Colab only:
import sys
if "google.colab" in sys.modules:
    !pip install datasets sacrebleu accelerate 
    exit() # Restart Colab

In [ ]:
#!pip install datasets sacrebleu accelerate
import logging

import numpy as np
import transformers
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

MODEL_CHECKPOINT = "Helsinki-NLP/opus-mt-tr-en"
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

metric = evaluate.load("sacrebleu")

def preprocess_function(examples):     

    prefix = ""
    max_input_length = 128
    max_target_length = 128
    source_lang = "tr"
    target_lang = "en"

    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result


logging.info(f"Transformers version: {transformers.__version__}")
raw_datasets = load_dataset("wmt16", "tr-en")
logging.info(raw_datasets["train"][0])

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

logging.info("Training arguments, including checkpoints ....")
batch_size = 16
model_name = MODEL_CHECKPOINT.split("/")[-1]
parameters = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-tr-to-en",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,  # set to 1 for experimentation. 1:30h per epoch on a T4 GPU
    predict_with_generate=True    
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model,
    parameters,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

logging.info("Finetuning training ....")
trainer.train()

trainer.save_model(f'model_output') 